In [15]:
import transformers
import json
import os
import torch
transformers.__version__

'4.16.0'

### Do some explore for the data

In [2]:
import pandas as pd

file_name = 'category_descriptions.csv'

df = pd.read_csv(file_name)

df.head()

,Category (incl. context and answer),Description,Answer Format,Group
0,Category: Document Name,Description: The name of the contract,Answer Format: Contract Name,Group: -
1,Category: Parties,Description: The two or more parties who signe...,Answer Format: Entity or individual names,Group: -
2,Category: Agreement Date,Description: The date of the contract,Answer Format: Date (mm/dd/yyyy),Group: 1
3,Category: Effective Date,Description: The date when the contract is eff...,Answer Format: Date (mm/dd/yyyy),Group: 1
4,Category: Expiration Date,Description: On what date will the contract's ...,Answer Format: Date (mm/dd/yyyy) / Perpetual,Group: 1


In [5]:
# what data type contains.
# this is all the supported clauses
types = [x.split(':')[-1].strip() for x in df['Category (incl. context and answer)'].values.tolist()]
print(types)

['Document Name', 'Parties', 'Agreement Date', 'Effective Date', 'Expiration Date', 'Renewal Term', 'Notice Period to Terminate Renewal', 'Governing Law', 'Most Favored Nation', 'Non-Compete', 'Exclusivity', 'No-Solicit of Customers', 'Competitive Restriction Exception', 'No-Solicit of Employees', 'Non-Disparagement', 'Termination for Convenience', 'Rofr/Rofo/Rofn', 'Change of Control', 'Anti-Assignment', 'Revenue/Profit Sharing', 'Price Restrictions', 'Minimum Commitment', 'Volume Restriction', 'IP Ownership Assignment', 'Joint IP Ownership', 'License Grant', 'Non-Transferable License', 'Affiliate License-Licensor', 'Affiliate License-Licensee', 'Unlimited/All-You-Can-Eat-License', 'Irrevocable or Perpetual License', 'Source Code Escrow', 'Post-Termination Services', 'Audit Rights', 'Uncapped Liability', 'Cap on Liability', 'Liquidated Damages', 'Warranty Duration', 'Insurance', 'Covenant Not to Sue', 'Third Party Beneficiary']


In [8]:
import json

data = json.load(open('CUADv1.json'))

len(data)

2

In [13]:
data['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [14]:
# there are 510 pdfs
len(data['data'])

510

In [2]:
# # create sample dataset
# n = 10

# def _load(f):
#     with open(f, 'r') as f:
#         return json.load(f)
    
# def _dump(js):
#     with open('sample.json', 'w') as f:
#         f.write(json.dumps(js))
        
# data = _load('CUADv1.json')

# data['data'] = data['data'][:n]

# _dump(data)
    

In [2]:
import json
def load_json(file_name):
    with open(file_name, 'r') as f:
        data = json.loads(f.read())
    return data


train_data = load_json('sample.json')

len(train_data)

2

```json
{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}```

In [9]:
len(train_data['data'][0]['paragraphs'][0].keys())

2

In [11]:
train_data['data'][0]['paragraphs'][0]['context']

'EXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company\'s  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.\n\n         B. Representations.  As an inducement to the Company to enter into this Agreement,  the  Distributor  has  represented  that  it has or  will  hav

In [116]:
train_data['data'][0]['paragraphs'][0]['qas']

[{'answers': [{'text': 'DISTRIBUTOR AGREEMENT', 'answer_start': 44}],
  'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name',
  'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
  'context': 'EXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company\'s  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other 

In [20]:
# let's create a func to make the real data with 
import copy

train_data = load_json('sample.json')

# def get_trained_data(train_data):
real_train_ds = []
for i, d in enumerate(train_data['data'][:2]):
    ps = d['paragraphs']
    for p in ps:
        context = p['context']
        for qas in p['qas']:
            qas['context'] = context
            qas.pop('is_impossible', '')
            if qas['answers'] == 0:
                continue
            if len(qas['answers'])  >= 1:
                # print(qas['answers'])
                tmp_ans = qas['answers'][0]
                tmp_ans['text'] = [tmp_ans['text']]
                tmp_ans['answer_start'] = [tmp_ans['answer_start']]
                qas['answers'] = tmp_ans
            
            # qas['answers']['text'] = [qas['answers']['text']]     
            real_train_ds.append(qas) 

    # return real_train_ds

# real_train_ds = get_trained_data(train_data)

len(real_train_ds)

82

In [21]:
real_train_ds[0]

{'answers': {'text': ['DISTRIBUTOR AGREEMENT'], 'answer_start': [44]},
 'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name',
 'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
 'context': 'EXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company\'s  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  p

In [24]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

ProxyError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-cased/resolve/main/added_tokens.json (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1112: The handshake operation timed out')))

In [23]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    # questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples['question'],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    print(inputs.keys())
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        print(sample_idx, answers)
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs
preprocess_training_examples(real_train_ds[0])


NameError: name 'tokenizer' is not defined

KeyError: 0

In [65]:
from torch.utils.data import DataLoader, Dataset


class MyData(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        data = self.data[i]
        return preprocess_training_examples(data)
    
dataset = MyData(real_train_ds)

loader = DataLoader(dataset, batch_size=2, shuffle=True)
    
    


In [66]:
for s in loader:
    print(s[0])
    break

TypeError: when tokenizing batches of text, `text_pair` must be a list or tuple with the same length as `text`.

In [36]:
train_data['data'][0]['paragraphs'][0]['qas'][0]

{'answers': [{'text': 'DISTRIBUTOR AGREEMENT', 'answer_start': 44}],
 'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name',
 'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
 'is_impossible': False}

In [5]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs